In [61]:
import collections
import http.client
import pandas as pd
from operator import itemgetter
import json
from collections import namedtuple
pd.set_option("display.max_rows", 200)
pd.set_option('display.max_colwidth', -1)

In [62]:
Rounds = namedtuple('Rounds', 'player round score strokes thru')

In [63]:
conn = http.client.HTTPConnection("api.sportradar.us")
conn.request("GET", "/golf-t2/leaderboard/pga/2018/tournaments/b404a8d5-5e33-4417-ae20-5d4d147042ee/leaderboard.json?api_key=u2rg6jhkezj6dkutjcmfqc8e")
res = conn.getresponse()
data = res.read()
d = data.decode("utf-8")

In [64]:
missed_cut = ["Sandy Lyle", "Charl Schwartzel", "Jose Maria Olazabal", "Jason Dufner", "Patrick Cantlay", 
              "Shubhankar Sharma", "Thomas Pieters", "Ted Potter", "Brendan Steele", "Dylan Frittelli", 
              "Danny Willett", "Ross Fisher", "Wesley Bryan", "Pat Perez", "Patton Kizzire", "Trevor Immelman", 
              "Joaquin Niemann", "Yuta Ikeda", "Alexander Noren", "Kevin Chappell", "Doc Redman", "Gary Woodland", 
              "Yusaku Miyazato", "Austin Cook", "Billy Horschel", "Ian Woosnam", "Mike Weir", "Larry Mize", 
              "Angel Cabrera", "Yuxin Lin", "Mark O'Meara", "Matt Parziale", "Sergio Garcia", "Harry Ellis"]
missed_cut_players = [p.lower().replace(" ", "") for p in missed_cut]
# sorted(missed_cut_players)

In [65]:
lb_df = pd.DataFrame(json.loads(d)['leaderboard'])
lb_df["full_name"] = lb_df["first_name"].str.lower().str.replace(" ", "") + lb_df["last_name"].str.lower().str.replace(" ", "")
lb_df[["first_name", "last_name", "position", "score", "strokes", "rounds", "full_name"]]
lb_df.index += 1
lb_df.to_csv("leaderboard.csv", columns=["first_name", "last_name", "position", "score", "strokes", "rounds", "full_name"])

In [49]:
def get_high_round(rounds):
    strokes = max(rounds, key=lambda x: x.strokes)
    return strokes
    
def get_score(player):
    player = player.lower().replace(" ", "")
    score = lb_df.loc[lb_df["full_name"] == player]["score"]
    strokes = lb_df.loc[lb_df["full_name"] == player]["strokes"]
    high_round = get_high_round(active_rounds)
    if player in missed_cut_players:
        return "**MC** Score: {} Strokes: {}".format(int(score) + high_round.score, int(strokes) + high_round.strokes)
    return "Score: {}, Strokes: {}".format(int(score), int(strokes))
    
def get_strokes(player):
    player = player.lower().replace(" ", "")
    high_strokes = 0
    if player in missed_cut_players:
        return int(lb_df.loc[lb_df["full_name"] == player]["strokes"]) + get_high_round(active_rounds).strokes
    else:
        return int(lb_df.loc[lb_df["full_name"] == player]["strokes"])

In [50]:
players_and_rounds = [("{} {}".format(r["first_name"], r["last_name"]), 
                       r["rounds"][2]) for r in json.loads(d)["leaderboard"] if len(r["rounds"]) > 2]
active_rounds = [Rounds(p[0], p[1]["sequence"], p[1]["score"], p[1]["strokes"], p[1]["thru"]) for p in players_and_rounds]

In [51]:
sorted(active_rounds, key=lambda x: x.strokes, reverse=True)

[Rounds(player='Vijay Singh', round=3, score=7, strokes=79, thru=18),
 Rounds(player='Brian Harman', round=3, score=4, strokes=76, thru=18),
 Rounds(player='Kyle Stanley', round=3, score=3, strokes=75, thru=18),
 Rounds(player='Chez Reavie', round=3, score=3, strokes=75, thru=18),
 Rounds(player='Zach Johnson', round=3, score=2, strokes=74, thru=18),
 Rounds(player='Rafael Cabrera-Bello', round=3, score=2, strokes=74, thru=18),
 Rounds(player='Branden Grace', round=3, score=2, strokes=74, thru=18),
 Rounds(player='Martin Kaymer', round=3, score=2, strokes=74, thru=18),
 Rounds(player='Doug Ghim', round=3, score=2, strokes=74, thru=18),
 Rounds(player='Phil Mickelson', round=3, score=2, strokes=74, thru=18),
 Rounds(player='Ian Poulter', round=3, score=2, strokes=74, thru=18),
 Rounds(player='Charley Hoffman', round=3, score=1, strokes=73, thru=18),
 Rounds(player='Fred Couples', round=3, score=1, strokes=73, thru=18),
 Rounds(player='Tyrrell Hatton', round=3, score=1, strokes=73, thru=

In [52]:
df = pd.read_csv("masters.csv")
df = df[["Pool Player", "Player1", "Player2", "Player3", "Player4", 
         "Player5", "Player6", "Player7", "Total", "Tie Breaker"]]

In [53]:
for index, row in df.iterrows():
    score_accum = 0
    for player in row.values[1:8]:
        score_accum += get_strokes(player)
    df.set_value(index, "Total", score_accum)

In [54]:
df = df.sort_values(by=['Total']).reset_index(drop=True)
for i in range(1,8):
    df["Score{}".format(i)] = df["Player{}".format(i)].apply(lambda x: get_score(x))

In [55]:
high_round = get_high_round(active_rounds)
print("High round of the day belongs to {}, with score: {}, strokes: {}, thru: {}".format(
    high_round.player, high_round.score, high_round.strokes, high_round.thru))

High round of the day belongs to Vijay Singh, with score: 7, strokes: 79, thru: 18


In [56]:
df.index += 1
df = df[["Pool Player", 
         "Player1", "Score1",
         "Player2", "Score2",
         "Player3", "Score3",
         "Player4", "Score4",
         "Player5", "Score5",
         "Player6", "Score6",
         "Player7", "Score7",
         "Total", "Tie Breaker"]]
df

,Pool Player,Player1,Score1,Player2,Score2,Player3,Score3,Player4,Score4,Player5,Score5,Player6,Score6,Player7,Score7,Total,Tie Breaker
1,Dave Peterson,Rory McIlroy,"Score: -10, Strokes: 195",Jason Day,"Score: -1, Strokes: 215",Patrick Reed,"Score: -13, Strokes: 187",Tony Finau,"Score: -1, Strokes: 211",Jason Dufner,**MC** Score: 13 Strokes: 229,Tiger Woods,"Score: 4, Strokes: 220",Martin Kaymer,"Score: 5, Strokes: 221",1478.0,278.0
2,Tristen Woods,Jon Rahm,"Score: -8, Strokes: 208",Henrik Stenson,"Score: -7, Strokes: 198",Patrick Reed,"Score: -13, Strokes: 187",Brendan Steele,**MC** Score: 14 Strokes: 230,Cameron Smith,"Score: -3, Strokes: 213",Billy Horschel,**MC** Score: 18 Strokes: 234,Adam Scott,"Score: 2, Strokes: 218",1488.0,279.0
3,Joe Kearney,Justin Thomas,"Score: -4, Strokes: 208",Jason Day,"Score: -1, Strokes: 215",Patrick Reed,"Score: -13, Strokes: 187",Louis Oosthuizen,"Score: -3, Strokes: 213",Thomas Pieters,**MC** Score: 14 Strokes: 230,Tiger Woods,"Score: 4, Strokes: 220",Russell Henley,"Score: 0, Strokes: 216",1489.0,279.0
4,Jeremy Szyba #3,Dustin Johnson,"Score: -5, Strokes: 203",Phil Mickelson,"Score: 7, Strokes: 223",Patrick Reed,"Score: -13, Strokes: 187",Branden Grace,"Score: 4, Strokes: 220",Adam Hadwin,"Score: 0, Strokes: 216",Daniel Berger,"Score: 2, Strokes: 218",Ian Poulter,"Score: 7, Strokes: 223",1490.0,278.0
5,Zachary Sight,Justin Thomas,"Score: -4, Strokes: 208",Phil Mickelson,"Score: 7, Strokes: 223",Patrick Reed,"Score: -13, Strokes: 187",Daniel Berger,"Score: 2, Strokes: 218",Siwoo Kim,"Score: 0, Strokes: 216",Tiger Woods,"Score: 4, Strokes: 220",Fred Couples,"Score: 3, Strokes: 219",1491.0,278.0
6,Paul Bower,Dustin Johnson,"Score: -5, Strokes: 203",Jason Day,"Score: -1, Strokes: 215",Bubba Watson,"Score: -6, Strokes: 210",Daniel Berger,"Score: 2, Strokes: 218",Adam Hadwin,"Score: 0, Strokes: 216",Tiger Woods,"Score: 4, Strokes: 220",Jimmy Walker,"Score: -1, Strokes: 215",1497.0,268.0
7,Joe Spallo,Rory McIlroy,"Score: -10, Strokes: 195",Jason Day,"Score: -1, Strokes: 215",Patrick Reed,"Score: -13, Strokes: 187",Kevin Chappell,**MC** Score: 16 Strokes: 232,Thomas Pieters,**MC** Score: 14 Strokes: 230,Adam Scott,"Score: 2, Strokes: 218",Bryson DeChambeau,"Score: 4, Strokes: 220",1497.0,277.0
8,Sam Montgomery,Jordan Spieth,"Score: -5, Strokes: 203",Tommy Fleetwood,"Score: -6, Strokes: 210",Charley Hoffman,"Score: -1, Strokes: 215",Branden Grace,"Score: 4, Strokes: 220",Adam Hadwin,"Score: 0, Strokes: 216",Jimmy Walker,"Score: -1, Strokes: 215",Bryson DeChambeau,"Score: 4, Strokes: 220",1499.0,282.0
9,Johnny Pendleton,Hideki Matsuyama,"Score: 0, Strokes: 216",Jason Day,"Score: -1, Strokes: 215",Patrick Reed,"Score: -13, Strokes: 187",Louis Oosthuizen,"Score: -3, Strokes: 213",Thomas Pieters,**MC** Score: 14 Strokes: 230,Tiger Woods,"Score: 4, Strokes: 220",Bryson DeChambeau,"Score: 4, Strokes: 220",1501.0,280.0
10,Chris Stine,Rory McIlroy,"Score: -10, Strokes: 195",Phil Mickelson,"Score: 7, Strokes: 223",Brian Harman,"Score: 7, Strokes: 223",Louis Oosthuizen,"Score: -3, Strokes: 213",Cameron Smith,"Score: -3, Strokes: 213",Tiger Woods,"Score: 4, Strokes: 220",Ryan Moore,"Score: 2, Strokes: 218",1505.0,280.0


In [57]:
df.to_csv("masters_results.csv")

# Tabula Fun

In [58]:
from tabula import read_pdf

In [59]:
df = read_pdf("/Users/jeremyszyba/Desktop/2018 Masters.pdf")
df = df.drop(df.index[10])
df

,Group 1,Group 2,Group 3,Group 4,Group 5,Field,Field.1,Field.2,Field.3
0,Dustin Johnson,Tommy Fleetwood,Bubba Watson,Branden Grace,Li Haotong,Ryan Armour,Mike Weir,Patton Kizzire,Lin Yuxin
1,Justin Thomas,Jason Day,Rafael Cabrera Bello,Louis Oosthuizen,Adam Hadwin,Daniel Berger,Bernd Wiesberger,Bernhard Langer,Doug Ghim
2,Jon Rahm,Paul Casey,Brian Harman,Patrick Cantlay,Kyle Stanley,Wesley Bryan,Danny Willett,Sandy Lyle,Russell Henley
3,Jordan Spieth,Alex Noren,Patrick Reed,Tony Finau,Cameron Smith,Angel Cabrera,Tiger Woods,Matt Parziale,Billy Horschel
4,Justin Rose,Henrik Stenson,Kevin Kisner,Ross Fisher,Thomas Pieters,Austin Cook,Ian Woosnam,Doc Redman,Larry Mize
5,Hideki Matsuyama,Marc Leishman,Xander Schauffele,Matthew Fitzpatrick,Satoshi Kodaira,Fred Couples,Yuta Ikdea,Ted Potter Jr.,Vijay Singh
6,Rory McIlroy,Tyrrell Hatton,Francesco Molinari,Daniel Berger,Dylan Frittelli,Bryson DeChambeau,Trevor Immelman,Charl Schwartzel,Jose Maria Olazabal
7,Rickie Fowler,Phil Mickelson,Charley Hoffman,Kevin Chappell,Chez Reavie,Harry Ellis,Zach Johnson,Adam Scott,Ryan Moore
8,Sergio Garcia,Pat Perez,Kiradech Aphibarnrat,Brendan Steele,Jason Dufner,Jhonattan Vegas,Martin Kaymer,Shubanhkar Sharma,Joaquin Niemann
9,Brooks Koepka,Matt Kuchar,Gary Woodland,Webb Simpson,Siwoo Kim,Jimmy Walker,Yusaku Miyazato,Webb Simpson,Mark O'Meara


In [60]:
print([item for item, count in collections.Counter(player_list).items() if count > 1] + ["brookskoepka", "ryanarmour"])

NameError: name 'player_list' is not defined